### Introducción
El objetivo de este proyecto es ayudar a la compañía OilyGiant a seleccionar las mejores ubicaciones para abrir 200 nuevos pozos petroleros en una región dada. Para ello, se ha utilizado un modelo predictivo basado en regresión lineal para estimar el volumen de reservas de crudo en diferentes regiones. Posteriormente, se han analizado los posibles beneficios de cada región, tomando en cuenta tanto el riesgo como la ganancia esperada. Finalmente, se ha implementado la técnica de bootstrapping para evaluar los riesgos de pérdida y determinar la mejor opción para invertir.

### Desarrollo
#### Paso 1: Cargar librerías y explorar dataset

In [1]:
#carga de librerías
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# se cargan los datos de cada región
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

# se usan las funciones info y head para inspección de la información  del dataset

print(geo_data_0.head())
print(geo_data_1.head())
print(geo_data_2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [3]:
geo_data_0.info()
geo_data_1.info()
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
#preprocesamiento
#revisión de filas duplicadas
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())
print()
#revisión de valores nulos
print(geo_data_0.isnull().sum())
print(geo_data_1.isnull().sum())
print(geo_data_2.isnull().sum())

0
0
0

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


No se encontraron duplicados ni valores faltantes en las columnas relevantes.

#### Paso 2: Generación de primera función y evaluación de rendimiento.
Se genera definición para la división de datos de entrenamiento y validación, entrenamiento del modelo de regresión lineal, generación de predicciones en el conjunto de validación y el calculo del RMSE para el modelo.

In [7]:
#se genera una definicion para utilizar en los 3 df
def train_and_validate_model(data):
    # se los datos en features (X) y target (y)
    X = data[['f0', 'f1', 'f2']]
    y = data['product']
    
    # se divide datos en grupos de entrenamiento y validación (75% - 25%)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=12345)
    
    # se entrena el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # se hacen predicciones en el conjunto de validación
    y_pred = model.predict(X_val)
    
    # calculo del RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    # resultados
    mean_pred = np.mean(y_pred)
    print(f"Volumen medio de reservas predicho: {mean_pred}")
    print(f"RMSE: {rmse}")
    
    return model, y_pred, y_val


Se evalua el rendimiento de cada región utilizando la definición para calcular el volumen medio de reservas predicho y el RMSE para cada DF.

In [8]:
# entrenamiento y validación del modelo para cada región
model_0, y_pred_0, y_val_0 = train_and_validate_model(geo_data_0)
model_1, y_pred_1, y_val_1 = train_and_validate_model(geo_data_1)
model_2, y_pred_2, y_val_2 = train_and_validate_model(geo_data_2)

Volumen medio de reservas predicho: 92.59256778438035
RMSE: 37.5794217150813
Volumen medio de reservas predicho: 68.728546895446
RMSE: 0.893099286775617
Volumen medio de reservas predicho: 94.96504596800489
RMSE: 40.02970873393434


#### Paso 3: Análisis de ganancias

Para cada una de las tres regiones, se calcula la ganancia potencial de los 200 pozos con mayor volumen de reservas predicho utilizando una nueva fórmula para calcular la ganancia basada en el ingreso por barril (4500 USD por cada 1000 barriles) y la cantidad mínima de reservas necesarias para que un pozo sea rentable (111 unidades o $500000 USD por pozo).


In [9]:
# ingreso por barril
income_per_barrel = 4500  # dólares por unidad (miles de barriles, literal $4500 USD = 1000 barriles = 1 unidad)
# se calcula la cantidad mínima de reservas necesarias para que un pozo sea rentable
min_reservoir_value = 500000 / income_per_barrel  # cantidad mínima de barriles (111 unidades aprox)

# Función para calcular la ganancia de un conjunto de pozos seleccionados
def calculate_profit(model, data, top_n=200):
    # predección del volumen de reservas
    X = data[['f0', 'f1', 'f2']]
    y_pred = model.predict(X)
    
    # selección de los 200 pozos con mayor volumen de reservas predicho
    data['predicted_reservoir'] = y_pred
    top_200 = data.nlargest(top_n, 'predicted_reservoir')
    
    # cálculo la ganancia potencial
    total_reservoir = top_200['predicted_reservoir'].sum() #suma volumen de reservas de los 200 pozos seleccionados
    total_profit = total_reservoir * income_per_barrel
    return total_profit, total_reservoir


In [10]:
# Se calculan las ganancias para las tres regiones con la función
profit_0, reservoir_0 = calculate_profit(model_0, geo_data_0)
profit_1, reservoir_1 = calculate_profit(model_1, geo_data_1)
profit_2, reservoir_2 = calculate_profit(model_2, geo_data_2)

print(f"Ganancia región 0: {profit_0} USD, Reservas: {reservoir_0} miles de barriles")
print(f"Ganancia región 1: {profit_1} USD, Reservas: {reservoir_1} miles de barriles")
print(f"Ganancia región 2: {profit_2} USD, Reservas: {reservoir_2} miles de barriles")

Ganancia región 0: 146895961.7696032 USD, Reservas: 32643.547059911823 miles de barriles
Ganancia región 1: 125246061.6171504 USD, Reservas: 27832.45813714453 miles de barriles
Ganancia región 2: 140637855.90942594 USD, Reservas: 31252.856868761322 miles de barriles


#### Paso 4: Evaluación de riesgos utilizando Bootstrapping
Se aplica la técnica de bootstrapping para simular la distribución de beneficios y calcular el intervalo de confianza del 95% para cada región. Además, se evalua el riesgo de pérdidas (cuando los beneficios sean negativos) para cada región.

In [11]:
def bootstrap(data, model, n_iterations=1000):
    profits = [] # se almacenará las ganancias obtenidas en cada iteración del remuestreo
    for _ in range(n_iterations):
        sample = data.sample(n=200, replace=True)
        total_profit, _ = calculate_profit(model, sample) # se calcula la ganancia total para la muestra usando la función calculate_profit
        profits.append(total_profit)
    
    # cálculo del intervalo de confianza del 95%
    lower_percentile = np.percentile(profits, 2.5)
    upper_percentile = np.percentile(profits, 97.5)
    
    # cálculo del riesgo de pérdidas
    losses = [p for p in profits if p < 0]
    risk_of_loss = len(losses) / len(profits) * 100 #porcentaje de iteraciones con pérdidas
    
    return np.mean(profits), lower_percentile, upper_percentile, risk_of_loss

# cálculo de riesgos y ganancias para cada región
mean_profit_0, ci_low_0, ci_up_0, risk_0 = bootstrap(geo_data_0, model_0)
mean_profit_1, ci_low_1, ci_up_1, risk_1 = bootstrap(geo_data_1, model_1)
mean_profit_2, ci_low_2, ci_up_2, risk_2 = bootstrap(geo_data_2, model_2)

print(f"Región 0: Ganancia promedio: {mean_profit_0}, Intervalo de confianza: ({ci_low_0}, {ci_up_0}), Riesgo de pérdidas: {risk_0}%")
print(f"Región 1: Ganancia promedio: {mean_profit_1}, Intervalo de confianza: ({ci_low_1}, {ci_up_1}), Riesgo de pérdidas: {risk_1}%")
print(f"Región 2: Ganancia promedio: {mean_profit_2}, Intervalo de confianza: ({ci_low_2}, {ci_up_2}), Riesgo de pérdidas: {risk_2}%")

Región 0: Ganancia promedio: 83314123.87808537, Intervalo de confianza: (80710679.4410337, 86282367.70441446), Riesgo de pérdidas: 0.0%
Región 1: Ganancia promedio: 61896170.1019992, Intervalo de confianza: (55864963.87042256, 67570377.80782768), Riesgo de pérdidas: 0.0%
Región 2: Ganancia promedio: 85467278.63676935, Intervalo de confianza: (82949927.46685341, 88106324.58971258), Riesgo de pérdidas: 0.0%


### Resultados y Conclusiones


**Resultados de los modelos**

Región 0:

- Ganancia potencial: 146,9 millones de USD
- Volumen de reservas: 32,643 miles de barriles
- RMSE: 37.58
- Riesgo de pérdidas: 0%

Región 1:

- Ganancia potencial: 125,2 millones de USD
- Volumen de reservas: 27,832 miles de barriles
- RMSE: 0.89
- Riesgo de pérdidas: 0%

Región 2:

- Ganancia potencial: 140,6 millones de USD
- Volumen de reservas: 31,253 miles de barriles
- RMSE: 40.03
- Riesgo de pérdidas: 0%

**Evaluación de riesgos utilizando bootstrapping**

Los resultados del bootstrapping revelaron lo siguiente:

Región 0:

- Ganancia promedio: 83.3 millones de USD
- Intervalo de confianza: (80.5 millones, 86.3 millones)
- Riesgo de pérdidas: 0%

Región 1:

- Ganancia promedio: 61.9 millones de USD
- Intervalo de confianza: (55.9 millones, 67.5 millones)
- Riesgo de pérdidas: 0%

Región 2:

- Ganancia promedio: 85.5 millones de USD
- Intervalo de confianza: (83.0 millones, 87.9 millones)
- Riesgo de pérdidas: 0%

**Conclusión**

Con base en los resultados obtenidos, la Región 2 se presenta como la mejor opción para abrir nuevos pozos petroleros. Aunque la Región 0 tiene una ganancia potencial ligeramente superior, la Región 2 muestra un riesgo de pérdidas nulo y un mayor beneficio promedio al considerar las variaciones en los ingresos debido a la técnica de bootstrapping. La Región 2 también presenta un intervalo de confianza más estrecho, lo que indica una mayor estabilidad en las predicciones del beneficio. Por lo tanto, la Región 2 es la más favorable para realizar la inversión, cumpliendo con los criterios de ganancia potencial, estabilidad en los resultados y un riesgo de pérdidas nulo.